<a href="https://colab.research.google.com/github/nupur710/Machine-Learning-in-Python/blob/master/Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy
!pip install scipy
!pip install scikit-learn
!pip install matplotlib
!pip install pandas
!pip install quandl

In [2]:
import pandas as pd
import quandl
import math
import numpy as np
from sklearn import preprocessing, model_selection, svm
from sklearn.linear_model import LinearRegression

In [3]:
df= quandl.get('WIKI/GOOGL')
print(df.head())

              Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
2004-08-19  100.01  104.06   95.96  100.335  44659000.0          0.0   
2004-08-20  101.01  109.08  100.50  108.310  22834300.0          0.0   
2004-08-23  110.76  113.48  109.05  109.400  18256100.0          0.0   
2004-08-24  111.24  111.60  103.57  104.870  15247300.0          0.0   
2004-08-25  104.76  108.00  103.88  106.000   9188600.0          0.0   

            Split Ratio  Adj. Open  Adj. High   Adj. Low  Adj. Close  \
Date                                                                   
2004-08-19          1.0  50.159839  52.191109  48.128568   50.322842   
2004-08-20          1.0  50.661387  54.708881  50.405597   54.322689   
2004-08-23          1.0  55.551482  56.915693  54.693835   54.869377   
2004-08-24          1.0  55.792225  55.972783  51.945350   52.597363   
2004-08-25          1.0  52.542193  54.167209  52.100830   53.1

In [4]:
df= df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]

In [5]:
print(df.head())

            Adj. Open  Adj. High   Adj. Low  Adj. Close  Adj. Volume
Date                                                                
2004-08-19  50.159839  52.191109  48.128568   50.322842   44659000.0
2004-08-20  50.661387  54.708881  50.405597   54.322689   22834300.0
2004-08-23  55.551482  56.915693  54.693835   54.869377   18256100.0
2004-08-24  55.792225  55.972783  51.945350   52.597363   15247300.0
2004-08-25  52.542193  54.167209  52.100830   53.164113    9188600.0


In [6]:
df['High PCT']= (df['Adj. High'] - df['Adj. Low'])/df['Adj. Low'] * 100.0

In [7]:
df['PCT Change']= (df['Adj. Close'] - df['Adj. Open'])/df['Adj. Open'] * 100.0

In [8]:
#features
df=  df[['Adj. Close', 'High PCT', 'PCT Change', 'Adj. Volume']]

In [9]:
print(df.head())

            Adj. Close  High PCT  PCT Change  Adj. Volume
Date                                                     
2004-08-19   50.322842  8.441017    0.324968   44659000.0
2004-08-20   54.322689  8.537313    7.227007   22834300.0
2004-08-23   54.869377  4.062357   -1.227880   18256100.0
2004-08-24   52.597363  7.753210   -5.726357   15247300.0
2004-08-25   53.164113  3.966115    1.183658    9188600.0


In [10]:
df.shape

(3424, 4)

In [11]:
df.fillna(value= 9999, inplace= True)

In [12]:
#label
forecast_col= 'Adj. Close'

In [17]:
#predict out 10% of dataset
forecast_out = math.ceil(0.01 * len(df))
forecast_out

35

In [ ]:
df

,Adj. Close,High PCT,PCT Change,Adj. Volume
Date,,,,
2004-08-19,50.322842,8.441017,0.324968,44659000.0
2004-08-20,54.322689,8.537313,7.227007,22834300.0
2004-08-23,54.869377,4.062357,-1.227880,18256100.0
2004-08-24,52.597363,7.753210,-5.726357,15247300.0
2004-08-25,53.164113,3.966115,1.183658,9188600.0
...,...,...,...,...
2018-03-21,1094.000000,1.976619,0.130884,1990515.0
2018-03-22,1053.150000,3.265882,-2.487014,3418154.0
2018-03-23,1026.550000,4.089299,-2.360729,2413517.0


In [19]:
#shifting so that the label column for each row is the prediction of Adj_Close forecast_out days in the future
df['label']= df[forecast_col].shift(-forecast_out)

In [ ]:
df

,Adj. Close,High PCT,PCT Change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,8.441017,0.324968,44659000.0,69.078238
2004-08-20,54.322689,8.537313,7.227007,22834300.0,67.839414
2004-08-23,54.869377,4.062357,-1.227880,18256100.0,68.912727
2004-08-24,52.597363,7.753210,-5.726357,15247300.0,70.668146
2004-08-25,53.164113,3.966115,1.183658,9188600.0,71.219849
...,...,...,...,...,...
2018-03-21,1094.000000,1.976619,0.130884,1990515.0,NaN
2018-03-22,1053.150000,3.265882,-2.487014,3418154.0,NaN
2018-03-23,1026.550000,4.089299,-2.360729,2413517.0,NaN


In [20]:
df.dropna(inplace=True)
df

,Adj. Close,High PCT,PCT Change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,8.441017,0.324968,44659000.0,69.078238
2004-08-20,54.322689,8.537313,7.227007,22834300.0,67.839414
2004-08-23,54.869377,4.062357,-1.227880,18256100.0,68.912727
2004-08-24,52.597363,7.753210,-5.726357,15247300.0,70.668146
2004-08-25,53.164113,3.966115,1.183658,9188600.0,71.219849
...,...,...,...,...,...
2018-01-30,1177.370000,1.142604,-0.029718,1792602.0,1094.000000
2018-01-31,1182.220000,1.213207,-0.134312,1643877.0,1053.150000
2018-02-01,1181.590000,1.547000,0.476195,2774967.0,1026.550000
